### Importacoes

In [1]:
# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import datetime #,  date

### Carregamento de dados, limpeza de colunas e linhas vazias e adequacao do tipo de dado

In [2]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/NovaListagem/listagem2016.xlsx'))

In [3]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

In [4]:
# Limpeza de linhas varias (NaN)
listagem = listagem.dropna()

In [5]:
listagem['Projeto'] = listagem['Projeto'].astype(int)
listagem['Ano'] = listagem['Ano'].astype(int)
listagem = listagem.rename(columns = {'Situação':'Situacao'})

### Criacao e preenchendo de novas colunas

In [ ]:
'''Criacao das colunas: 
codigo do tipo
id do projeto
data de proposicao do projeto
status da tramitacao
conferência da situacao
'''
listagem['Codigo_Tipo'] = ''
listagem['Id_Projeto_Camara'] = ''
listagem['Data_Projeto'] = ''
listagem['Tramitacao_Encerrada'] = ''
listagem['Aprovado'] = '' 

#listagem['Tramitacao'] = '' # UNIAO DOS DOIS CAMPOS ANTERIORES EM REVISAO

In [ ]:
# Preenchimento do campo Codigo_Tipo
unknown_count = 0

for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Codigo_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Codigo_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Codigo_Tipo'][i] = 'REP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Codigo_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Codigo_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Codigo_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Codigo_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Codigo_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Codigo_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Codigo_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Codigo_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Codigo_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Codigo_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Codigo_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Codigo_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Codigo_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Codigo_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Codigo_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Codigo_Tipo'][i] = 'RCPI'
    elif(listagem["Tipo"][i] == "Pedido de Informação FiscalizaJF"):
        listagem['Codigo_Tipo'][i] = 'PEDFF'
    elif(listagem["Tipo"][i] == "Emenda"):
        listagem['Codigo_Tipo'][i] = 'EMENDA'
    else:
        listagem['Codigo_Tipo'][i] = 'DESCONHECIDO'
        unknown_count += 1

print(f'Fim do processo. Traducao realizada com {unknown_count} registros nao encontrados')

### Importacoes

In [ ]:
# Preenchimento do campo Id_Projeto_Camara
count = 1
failed_req= 0
failed = False
id_projeto_camara = ''
max_failed_req = 3
notfound_count = 0
req_count = 0
max_req = 1500
export = False

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                id_projeto_camara = ''
                failed = False
                # Variaveis de requisicao
                num_proj = int(listagem['Projeto'][i])
                ano_proj = int(listagem['Ano'][i])
                tipo_proj = listagem['Codigo_Tipo'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                    #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    req_count+=1
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', ''))
                        id_projeto_camara = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        try:
                            retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                            id_projeto_camara = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                        except:
                            id_projeto_camara = 'Nao Encontrado'
                            notfound_count += 1
                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)

                listagem['Id_Projeto_Camara'][i] = id_projeto_camara
                #resposta.close()
                session.close()
                time.sleep(0.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas ou houve mais de 5 mil requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break

    if failed_req > max_failed_req or req_count > max_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

now =datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
    # Exportacao da listagem no estado atual para arquivo csv
    file = 'ListagemExport2012-2015_checkpoint_' + dthr_finalizado
    path_to_file = '../ListagemExp/'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

except:
    export = False


if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')


In [ ]:
# Preenchimento do campo Data_Projeto, Tramitacao_Encerrada e Aprovado
count = 1
failed_req= 0
failed = False
max_failed_req = 2
req_count = 0
max_req = 5
export = False

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto_Camara'][i] != '': 
            with requests.Session() as session:
                data_projeto = ''
                tramitacao_encerrada = ''
                aprovado = ''

                failed = False
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto_Camara'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
#                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', ''))
                    req_count+=1

                    #Identificacao da data de registro do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                    except:
                        data_projeto = 'Data nao encontrada'
                        notfound_count_dt += 1


                    #Identificacao do encerramento da tramitacao do projeto
                    if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jur' in listagem['Situacao'][i]:    
                        tramitacao_encerrada = 'SIM'
                    else:
                        try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                        
                            if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                                tramitacao_encerrada = 'SIM'
                            else:
                                tramitacao_encerrada = 'NAO'
                        except:
                            tramitacao_encerrada = 'Informacao nao encontrada'
                            notfound_count_tram+=1


                    #Identificacao da aprovacao do projeto
                    if tramitacao_encerrada == 'NAO': 
                        aprovado = 'TRAMITACAO'
                    elif 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jur' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                    elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetad' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                    #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                    #    if listagem['Situacao'][i] == 'Arquivada': 
                    #        aprovado = 'NAO'        
                    #    else:        
                    #        aprovado = 'SIM'
                    #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                    else:
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                            if(re.search('APROVAD', retorno)):
                                aprovado = 'SIM'
                            else:
                                aprovado = 'NAO'
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1

                except:
                    failed = True
                    time.sleep(180)

                listagem['Data_Projeto'][i] = data_projeto
                listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
                listagem['Aprovado'][i] = aprovado

                #resposta.close()
                session.close()
                time.sleep(0.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req or req_count > max_req:
            break
            
    if failed_req > max_failed_req or req_count > max_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Tramitacao_Encerrada'][i] == '' and listagem['Aprovado'][i] == '' and listagem['Id_Projeto_Camara'][i] != '':
            count +=1

now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
    # Exportacao da listagem no estado atual para arquivo csv-
    file = 'ListagemExport2012-2015_checkpoint_' + dthr_finalizado
    path_to_file = '../ListagemExp/'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

except:
    export = False

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')


### Exportando informacoes sobre projetos e autoria para arquivos CSV

In [7]:
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport_2012-2015.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Codigo_Tipo'
               , 'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']
              #, 'Autor'] 
    
if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

In [8]:
# Logica de identificador de autor (Id_Autor) e identificacao de autoria, separando casos de 'multiautoria'
autoria_df_estrutura = {'nome_autor': [''],'id_projeto_camara': ['']}
autoria_df = pd.DataFrame(columns=['nome_autor','id_projeto_camara'])
autores = ['']
aut = ['']

for i in listagem.index:
    aut = listagem['Autor'][i].split(', ')
    
    
    for item in aut:
        autoria = pd.DataFrame({'nome_autor': item, 
                      'id_projeto_camara': listagem['Id_Projeto_Camara'][i]}, index=[0]) 
        autoria_df = autoria_df.append(autoria, ignore_index=True, verify_integrity=False, sort=None) 

# Exportacao da listagem de autorias para arquivo csv
path_to_file = '../ListagemExp/AutoriasExport_2012-2015.csv'
path = Path(path_to_file)
exp_columns = ["id_projeto_camara", "nome_autor"]

if path.is_file():
    exp_header = False
else:    
    exp_header = True

autoria_df.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

## Visualizacao do dataframe e testes

In [ ]:
display(listagem)

In [ ]:
'''
  
# Contagem e conferencia de qual eh o ultimo projeto com Id_Projeto_Camara preenchido 
count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto_Camara'][i] == "":
        print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto_Camara'][i])
        print(count)
        break
    else:
        count +=1
'''

In [ ]:
'''
# Conferencia de valores
i = 936

print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], 
      listagem['Id_Projeto_Camara'][i], listagem['Data_Projeto'][i], listagem['Situacao'][i])

#https://www.camarajf.mg.gov.br/sal/proposicao.php?num=45773
'''

In [ ]:
'''
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport.csv'
path = Path(path_to_file)

#exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Codigo_Tipo', 
#               'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header) 
'''

### Criacao de 'checkpoint' e recuperacao de trabalho parado - especialmente em caso de endpoint fora do ar

In [ ]:
# Exportacao da listagem no estado atual para arquivo csv
now =datetime.now()

file = 'ListagemExport2012-2015_checkpoint_' + now.strftime("%d-%m-%Y_%H-%M-%S")
path_to_file = '../ListagemExp/'+file+'.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
               'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 
print(path)

In [4]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = '..//ListagemExp/ListagemExport2012-2015_checkpoint_05-11-2022_20-29-57.csv'
listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#listagem = listagem.rename(columns = {"Id_Projeto":"Id_Projeto_Camara"})
#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [5]:
count=0
for i in listagem.index:
    if listagem['Codigo_Tipo'][i] == 'EMENDA':
        listagem['Aprovado'] = 'Nao encontrado'
        listagem['Data_Projeto'] = 'Data nao encontrada'
        listagem['Tramitacao_Encerrada'] = 'Nao encontrado'
        count+=1
print(count)

0


### Identificacao do estado da importacao dos dados

In [6]:
tem=0
naovaiter=0
naotem=0

#oid = ''
for i in listagem.index:
    if listagem['Id_Projeto_Camara'][i]!='' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
#    if listagem['Data_Projeto'][i]!='':
#    if listagem['Codigo_Tipo'][i]!='EMENDA':
#    if (listagem['Data_Projeto'][i] != '' or listagem['Tramitacao_Encerrada'][i] != '' or listagem['Aprovado'][i] != ''):# and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
        tem+=1
    elif listagem['Codigo_Tipo'][i] in ('DESCONHECIDO','EMENDA'):
        naovaiter+=1
    else:
        naotem+=1
        #oid=i
print(tem)
print(naotem)
print(naovaiter)
#print(oid)

12775
0
0


In [17]:
# Teste de requisicao isolada

with requests.Session() as session:
    #num_proj = int(listagem['Projeto'][i])
    #ano_proj = int(listagem['Ano'][i])
    #tipo_proj = listagem['Codigo_Tipo'][i]
    
    query = {"numero": 22, "ano": 2017, "tipo": 'PEDIF' 
            , "autor": 'Mesa Diretora - biênio 2017/2018'
            }
    #query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}    
    try:
        # Requisicao para o portal de propostas da Camara
        
        resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
        #print(resposta.headers)
        #print(resposta.json())
        #print(resposta.status_code)
    except:
        print('Falhou')
# You can inspect the response just like you did before
print(resposta.content)

b'<html>\r\n\r\n<head>\r\n<title>\r\nSistema de Apoio Legislativo</title>\r\n<link rel="stylesheet" type="text/css" href="/sal/config.css"></head>\r\n\r\n<body bgcolor="F4F4EA" leftmargin="0" topmargin="0" marginwidth="0" marginheight="0">\r\n  <table width="100%" height="100%" border="0" cellpadding="0" cellspacing="0">\r\n    <tr>\r\n      <td align="center" valign="top">\r\n        <table width="770" border="0" cellpadding="1" cellspacing="0">\r\n          <tr>\r\n            <td valign="top" bgcolor="A2A88A"> \r\n              <TABLE WIDTH="1000" BORDER="0" CELLPADDING="0" CELLSPACING="0" bgcolor="#FFFFFF">\r\n                <TR> \r\n                  <TD width="1000" valign="top">\r\n\t\t\t\t    <div align="center"><img src="img/cabecalho_novo.png" />                  </div></TD>\r\n                </TR>\r\n                \r\n\t\t\r\n                <tr bgcolor="FFFFFF"><td height="5"></td></tr>\r\n                <tr><td valign="top">\r\n\r\n<table width="100%" bgcolor="F1F2EE"

In [8]:
count=0
for i in listagem.index:
    if listagem['Id_Projeto_Camara'][i] == 68685  and listagem['Ementa'][i] == 'Solicita que nos respondam o porquê das áreas do bairro Novo Triunfo II, mais precisamente os CEPs 36092270 / 36092271 / 36092272 / 36092273 / 36092274 / 36092275 / 36092276 e 36092277, não estarem sendo atendidas pelo setor responsável pelo distribuição de correspondências na zona norte em Juiz de Fora.':
        count+=1
       # listagem['Id_Projeto_Camara'][i] = 68348
       # listagem['Data_Projeto'][i] = ''
       # listagem['Tramitacao_Encerrada'][i] = ''
       # listagem['Aprovado'][i] = ''

print(count)


KeyError: 'Id_Projeto_Camara'

In [ ]:
count=0
for i in listagem.index:
    if listagem['Codigo_Tipo'][i] == 'EMENDA':
        count-=1
        listagem['Id_Projeto_Camara'][i] = count
       # listagem['Data_Projeto'][i] = ''
       # listagem['Tramitacao_Encerrada'][i] = ''
       # listagem['Aprovado'][i] = ''

print(count)
